# 📊 Análisis Exploratorio de Datos - SmartHabits

**Objetivo**: Realizar un análisis exploratorio completo de los datos de hábitos sostenibles para entender patrones, distribuciones y relaciones en el comportamiento de los usuarios.

**Autor**: Irving Morales Domínguez (220732)  
**Proyecto**: SmartHabits - Sistema de Hábitos Sostenibles  
**Fecha**: Enero 2025

---


## 📚 Importación de Librerías


In [ ]:
# Librerías básicas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings

# Librerías para conexión a API/Base de datos
import requests
import json
from supabase import create_client, Client
from datetime import datetime, timedelta
import os
from dotenv import load_dotenv

# Configuración
warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Configuración de visualización
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Librerías importadas correctamente")


## 🗂️ Conexión a Datos Reales de SmartHabits

Vamos a conectarnos a la API de SmartHabits y a la base de datos Supabase para obtener los datos reales de usuarios y sus hábitos sostenibles.


In [ ]:
# Cargar variables de entorno
load_dotenv()

# Configuración de conexión a Supabase
SUPABASE_URL = os.getenv('SUPABASE_URL', 'https://your-project.supabase.co')
SUPABASE_KEY = os.getenv('SUPABASE_ANON_KEY', 'your-anon-key')

# Configuración de la API de SmartHabits
API_BASE_URL = os.getenv('SMARTHABITS_API_URL', 'http://localhost:3000/api')

print("🔧 Configuración de conexiones:")
print(f"   • Supabase URL: {SUPABASE_URL}")
print(f"   • API Base URL: {API_BASE_URL}")

# Función para conectar a Supabase
def connect_to_supabase():
    try:
        supabase: Client = create_client(SUPABASE_URL, SUPABASE_KEY)
        return supabase
    except Exception as e:
        print(f"❌ Error conectando a Supabase: {e}")
        return None

# Función para obtener datos de la API
def fetch_from_api(endpoint):
    try:
        response = requests.get(f"{API_BASE_URL}/{endpoint}")
        if response.status_code == 200:
            return response.json()
        else:
            print(f"❌ Error en API {endpoint}: {response.status_code}")
            return None
    except Exception as e:
        print(f"❌ Error conectando a API: {e}")
        return None

print("✅ Funciones de conexión configuradas")


In [ ]:
# Intentar obtener datos reales de SmartHabits
print("🔄 Intentando conectar a datos reales...")

# Inicializar cliente de Supabase
supabase = connect_to_supabase()
use_real_data = False

if supabase:
    try:
        # Obtener usuarios
        users_response = supabase.table('users').select('*').execute()
        
        # Obtener hábitos/actividades
        habits_response = supabase.table('user_habits').select('*').execute()
        
        # Obtener categorías de hábitos
        categories_response = supabase.table('habit_categories').select('*').execute()
        
        if users_response.data and habits_response.data:
            print("✅ Datos reales obtenidos de Supabase")
            
            # Convertir a DataFrames
            users_df = pd.DataFrame(users_response.data)
            habits_df = pd.DataFrame(habits_response.data)
            categories_df = pd.DataFrame(categories_response.data)
            
            use_real_data = True
            
            print(f"   • Usuarios: {len(users_df)}")
            print(f"   • Registros de hábitos: {len(habits_df)}")
            print(f"   • Categorías: {len(categories_df)}")
            
        else:
            print("⚠️ No se encontraron datos en Supabase")
            
    except Exception as e:
        print(f"Error obteniendo datos de Supabase: {e}")

# Intentar API como alternativa
if not use_real_data:
    print("🔄 Intentando API de SmartHabits...")
    
    users_data = fetch_from_api('users')
    habits_data = fetch_from_api('habits')
    
    if users_data and habits_data:
        users_df = pd.DataFrame(users_data)
        habits_df = pd.DataFrame(habits_data)
        use_real_data = True
        print("✅ Datos obtenidos de API")
        print(f"   • Usuarios: {len(users_df)}")
        print(f"   • Hábitos: {len(habits_df)}")

print(f"🎯 Usando datos {'reales' if use_real_data else 'simulados'}")


In [ ]:
# Fallback: Generar datos simulados si no hay conexión a datos reales
if not use_real_data:
    print("📊 Generando datos simulados para demostración...")
    
    # Configuración de semilla para reproducibilidad
    np.random.seed(42)
    
    # Parámetros de simulación
    n_users = 1000
    n_days = 90  # 3 meses de datos
    
    # Categorías de hábitos sostenibles de SmartHabits
    habit_categories = [
        'reciclaje', 'transporte_sostenible', 'ahorro_energia', 
        'consumo_agua', 'alimentacion_sostenible', 'compras_responsables'
    ]
    
    # Generar datos de usuarios simulados
    users_data = []
    for user_id in range(1, n_users + 1):
        # Perfil de usuario
        user_type = np.random.choice(['principiante', 'intermedio', 'avanzado'], p=[0.4, 0.4, 0.2])
        age_group = np.random.choice(['18-25', '26-35', '36-45', '46+'], p=[0.3, 0.4, 0.2, 0.1])
        
        # Nivel de actividad base según tipo de usuario
        if user_type == 'principiante':
            activity_base = np.random.uniform(0.1, 0.4)
        elif user_type == 'intermedio':
            activity_base = np.random.uniform(0.4, 0.7)
        else:  # avanzado
            activity_base = np.random.uniform(0.7, 0.95)
        
        users_data.append({
            'user_id': user_id,
            'user_type': user_type,
            'age_group': age_group,
            'activity_base': activity_base,
            'created_at': datetime.now() - timedelta(days=np.random.randint(30, 180))
        })
    
    users_df = pd.DataFrame(users_data)
    
    # Generar datos de actividad diaria por hábito
    habits_data = []
    for _, user in users_df.iterrows():
        user_id = user['user_id']
        activity_base = user['activity_base']
        
        # Generar preferencias por categoría de hábito
        category_preferences = np.random.dirichlet(np.ones(len(habit_categories)) * 2)
        
        for day in range(n_days):
            # Variación temporal
            day_of_week = day % 7
            weekend_factor = 0.7 if day_of_week >= 5 else 1.0
            time_trend = 1 + (day / n_days) * 0.3
            
            for i, category in enumerate(habit_categories):
                prob = activity_base * category_preferences[i] * weekend_factor * time_trend
                prob = min(prob, 0.95)
                
                completed = np.random.binomial(1, prob)
                
                if completed:
                    base_points = {'reciclaje': 10, 'transporte_sostenible': 15, 'ahorro_energia': 12,
                                  'consumo_agua': 8, 'alimentacion_sostenible': 20, 'compras_responsables': 25}
                    
                    points = base_points[category] * np.random.uniform(0.8, 1.2)
                    co2_saved = points * np.random.uniform(0.05, 0.15)
                    
                    habits_data.append({
                        'user_id': user_id,
                        'habit_category': category,
                        'completed_date': datetime.now() - timedelta(days=n_days-day),
                        'points_earned': round(points, 2),
                        'co2_saved_kg': round(co2_saved, 3),
                        'day_of_week': day_of_week,
                        'completed': True
                    })
    
    habits_df = pd.DataFrame(habits_data)
    
    print(f"Datos simulados generados:")
    print(f"   • Usuarios: {len(users_df)}")
    print(f"   • Registros de hábitos: {len(habits_df)}")
    print(f"   • Actividades completadas: {habits_df['completed'].sum():,}")

# Mostrar información básica de los datos obtenidos
print("\\n=== INFORMACIÓN DE DATOS ===")
print(f"Fuente: {'Datos reales de SmartHabits' if use_real_data else 'Datos simulados'}")
print(f"Usuarios únicos: {users_df['user_id'].nunique():,}")
if 'habit_category' in habits_df.columns:
    print(f"Categorías de hábitos: {habits_df['habit_category'].nunique()}")
print(f"Total registros de actividad: {len(habits_df):,}")
